In [1]:
!wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/part-of-speech/pos-data-v3.json

--2019-09-11 07:47:07--  https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/part-of-speech/pos-data-v3.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3085086 (2.9M) [text/plain]
Saving to: ‘pos-data-v3.json’

pos-data-v3.json    100%[===================>]   2.94M  --.-KB/s    in 0.05s   

2019-09-11 07:47:08 (63.2 MB/s) - ‘pos-data-v3.json’ saved [3085086/3085086]



In [2]:
import json

with open('pos-data-v3.json') as fopen:
  pos = json.load(fopen)
  
pos[0], len(pos)

(['Sampul', 'sampul', 'NOUN'], 103695)

In [0]:
pos = pos[:10000]

In [4]:
pos[:10]

[['Sampul', 'sampul', 'NOUN'],
 ['dari', 'dari', 'ADP'],
 ['dua', 'dua', 'NUM'],
 ['singel', 'singel', 'NOUN'],
 ['pertama', 'pertama', 'NUM'],
 ['difoto', 'difoto', 'VERB'],
 ['oleh', 'oleh', 'ADP'],
 ['Emma', 'emma', 'PROPN'],
 ['Summerton', 'summerton', 'PROPN'],
 ['pada', 'pada', 'ADP']]

In [0]:
import re

def process_string(string):
    string = re.sub('[^A-Za-z0-9\-\/ ]+', ' ', string).split()
    return [to_title(y.strip()) for y in string]
  
def to_title(string):
    if string.isupper():
        string = string.title()
    return string

In [6]:
process_string('H')

['H']

In [7]:
texts, labels = [], []
for i in pos:
    try:
        texts.append(process_string(i[0])[0])
        labels.append(i[-1])
    except Exception as e:
        print(e, i)

list index out of range ['%', '%', 'SYM']
list index out of range ['%', '%', 'SYM']
list index out of range ['*', '*', 'SYM']
list index out of range ['뭘봐', '뭘봐', 'PROPN']
list index out of range ['%', '%', 'SYM']
list index out of range ['ひ', 'ひ', 'PROPN']
list index out of range ['ヒ', 'ヒ', 'PROPN']
list index out of range ['形聲', '形聲', 'NOUN']
list index out of range ['°', '°', 'SYM']
list index out of range ['汉', '汉', 'PROPN']
list index out of range ['东', '东', 'PROPN']
list index out of range ['王', '王', 'PROPN']
list index out of range ['（', '（', 'PROPN']
list index out of range ['伊', '伊', 'PROPN']
list index out of range ['）', '）', 'PROPN']
list index out of range ['ȝ', 'ȝ', 'PROPN']
list index out of range ['%', '%', 'SYM']
list index out of range ['°', '°', 'SYM']


In [8]:
texts[-10:], labels[-10:]

(['mahasiswa',
  'Apakah',
  'ada',
  'perkembangan',
  'perbaikan',
  'Platz',
  'kemudian',
  'kembali',
  'kepada',
  'rancangan'],
 ['NOUN',
  'ADV',
  'VERB',
  'NOUN',
  'NOUN',
  'PROPN',
  'ADV',
  'VERB',
  'ADP',
  'NOUN'])

In [0]:
import numpy as np

word2idx = {'PAD': 0,'UNK':1}
tag2idx = {'PAD': 0}
char2idx = {'PAD': 0}
word_idx = 2
tag_idx = 1
char_idx = 1

def parse_XY(texts, labels):
    global word2idx, tag2idx, char2idx, word_idx, tag_idx, char_idx
    X, Y, sentences = [], [], []
    for no, text in enumerate(texts):
        tag = labels[no]
        for c in text:
            if c not in char2idx:
                char2idx[c] = char_idx
                char_idx += 1
        if tag not in tag2idx:
            tag2idx[tag] = tag_idx
            tag_idx += 1
        Y.append(tag2idx[tag])
        if text not in word2idx:
            word2idx[text] = word_idx
            word_idx += 1
        X.append(word2idx[text])
        sentences.append(text)
    return X, np.array(Y), sentences

In [0]:
X, Y, sentences = parse_XY(texts, labels)

In [0]:
seq_len = 50
def iter_seq(x):
    return np.array([x[i: i+seq_len] for i in range(0, len(x)-seq_len, 1)])

def to_train_seq(*args):
    return [iter_seq(x) for x in args]

def generate_char_seq(batch):
    x = [[len(i) for i in k] for k in batch]
    len_maxlen = max([len(i) for i in x])
    maxlen = max([j for i in x for j in i])
    temp = np.zeros((len(batch), len_maxlen,maxlen),dtype=np.int32)
    for i in range(len(batch)):
        for k in range(len_maxlen):
            if k < len(batch[i]):
                for no, c in enumerate(batch[i][k][::-1]):
                    temp[i,k,-1-no] = char2idx[c]
    return temp
  
def word_seq(batch):
  temp = np.zeros((batch.shape[0], batch.shape[1]),dtype=np.int32)
  for i in range(batch.shape[0]):
    row = batch[i][::-1]
    for no in range(batch.shape[1]):
      temp[i,-1-no] = word2idx.get(row[no], 1)
  return temp

In [0]:
idx2word={idx: tag for tag, idx in word2idx.items()}
idx2tag = {i: w for w, i in tag2idx.items()}

In [13]:
X_seq, Y_seq = to_train_seq(sentences, Y)
X_char_seq = generate_char_seq(X_seq)
X_seq.shape, X_char_seq.shape

((9932, 50), (9932, 50, 23))

In [14]:
text_sample = ['Husein makan ayam', 'Husein suka ayam dan itik']
text_sample = [s.split() for s in text_sample]
text_sample

[['Husein', 'makan', 'ayam'], ['Husein', 'suka', 'ayam', 'dan', 'itik']]

In [15]:
generate_char_seq(text_sample).shape

(2, 5, 6)

In [0]:
X = word_seq(X_seq)

In [17]:
Y_seq[0]

array([1, 2, 3, 1, 3, 4, 2, 5, 5, 2, 1, 5, 3, 6, 3, 1, 7, 4, 2, 7, 8, 4,
       2, 4, 1, 2, 1, 5, 5, 7, 1, 4, 5, 9, 4, 7, 2, 1, 1, 1, 4, 2, 1, 7,
       2, 5, 5, 6, 4, 2])

In [18]:
idx2tag[1]

'NOUN'

In [19]:
idx2word[2]

'Sampul'

In [20]:
X[0]

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19,  8, 20, 21, 22, 23, 24, 25, 26, 12, 27,  2, 28, 25, 29, 30, 31,
       32, 33, 26, 34, 35, 36, 37, 26, 38, 39,  8, 40, 41, 15, 22, 11],
      dtype=int32)

In [0]:
from sklearn.model_selection import train_test_split

train_X, test_X, char_train_X, char_test_X, train_Y, test_Y = train_test_split(X, 
                                                                               X_char_seq, 
                                                                               Y_seq, 
                                                                               test_size = 0.2)

In [0]:
class Model:
    def __init__(
        self,
        dim_word,
        dim_char,
        dropout,
        learning_rate,
        hidden_size_char,
        hidden_size_word,
        num_layers,
    ):
        def cells(size, reuse = False):
            return tf.contrib.rnn.DropoutWrapper(
                tf.nn.rnn_cell.LSTMCell(
                    size,
                    initializer = tf.orthogonal_initializer(),
                    reuse = reuse,
                ),
                state_keep_prob = dropout,
                output_keep_prob = dropout,
            )
          
        self.word_ids = tf.placeholder(tf.int32, shape = [None, None])
        self.char_ids = tf.placeholder(tf.int32, shape = [None, None, None])
        self.labels = tf.placeholder(tf.int32, shape = [None, None])
        self.maxlen = tf.shape(self.word_ids)[1]
        self.lengths = tf.count_nonzero(self.word_ids, 1)

        self.word_embeddings = tf.Variable(
            tf.truncated_normal(
                [len(word2idx), dim_word], stddev = 1.0 / np.sqrt(dim_word)
            )
        )
        self.char_embeddings = tf.Variable(
            tf.truncated_normal(
                [len(char2idx), dim_char], stddev = 1.0 / np.sqrt(dim_char)
            )
        )
        
        word_embedded = tf.nn.embedding_lookup(
            self.word_embeddings, self.word_ids
        )
        char_embedded = tf.nn.embedding_lookup(
            self.char_embeddings, self.char_ids
        )
        s = tf.shape(char_embedded)
        char_embedded = tf.reshape(
            char_embedded, shape = [s[0] * s[1], s[-2], dim_char]
        )
        
        for n in range(num_layers):
            (out_fw, out_bw), (
                state_fw,
                state_bw,
            ) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cells(hidden_size_char),
                cell_bw = cells(hidden_size_char),
                inputs = char_embedded,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_char_%d' % (n),
            )
            char_embedded = tf.concat((out_fw, out_bw), 2)
            
        output = tf.reshape(
            char_embedded[:, -1], shape = [s[0], s[1], 2 * hidden_size_char]
        )
        word_embedded = tf.concat([word_embedded, output], axis = -1)
        for n in range(num_layers):
            (out_fw, out_bw), (
                state_fw,
                state_bw,
            ) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cells(hidden_size_word),
                cell_bw = cells(hidden_size_word),
                inputs = word_embedded,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_word_%d' % (n),
            )
            word_embedded = tf.concat((out_fw, out_bw), 2)
            
        logits = tf.layers.dense(word_embedded, len(idx2tag))
        y_t = self.labels
        log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
            logits, y_t, self.lengths
        )
        self.cost = tf.reduce_mean(-log_likelihood)
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        mask = tf.sequence_mask(self.lengths, maxlen = self.maxlen)
        self.tags_seq, tags_score = tf.contrib.crf.crf_decode(
            logits, transition_params, self.lengths
        )
        self.tags_seq = tf.identity(self.tags_seq, name = 'logits')
        
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(self.tags_seq, mask)
        mask_label = tf.boolean_mask(y_t, mask)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [23]:
import tensorflow as tf

tf.reset_default_graph()
sess = tf.InteractiveSession()

dim_word = 128
dim_char = 256
dropout = 1.0
learning_rate = 1e-3
hidden_size_char = 64
hidden_size_word = 64
num_layers = 2
batch_size = 64

model = Model(dim_word,dim_char,dropout,learning_rate,hidden_size_char,hidden_size_word,num_layers)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call i

In [24]:
import time
from tqdm import tqdm

for e in range(3):
    lasttime = time.time()
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_char = char_train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.word_ids: batch_x,
                model.char_ids: batch_char,
                model.labels: batch_y
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    pbar = tqdm(
        range(0, len(test_X), batch_size), desc = 'test minibatch loop'
    )
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_char = char_test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.word_ids: batch_x,
                model.char_ids: batch_char,
                model.labels: batch_y
            },
        )
        assert not np.isnan(cost)
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (e, train_loss, train_acc, test_loss, test_acc)
    )

train minibatch loop:   0%|          | 0/125 [00:00<?, ?it/s]

time taken: 82.04590725898743
epoch: 0, training loss: 84.115309, training acc: 0.465857, valid loss: 20.123173, valid acc: 0.941000



train minibatch loop:   0%|          | 0/125 [00:00<?, ?it/s]

time taken: 80.70353841781616
epoch: 1, training loss: 6.858737, training acc: 0.980118, valid loss: 2.518778, valid acc: 1.024097



test minibatch loop: 100%|██████████| 32/32 [00:09<00:00,  3.44it/s, accuracy=1, cost=0.442]

time taken: 80.36205554008484
epoch: 2, training loss: 1.285965, training acc: 1.004070, valid loss: 0.719654, valid acc: 1.029562



In [0]:
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p])
        out.append(out_i)
    return out

In [26]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_char = char_test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    predicted = pred2label(sess.run(model.tags_seq,
            feed_dict = {
                model.word_ids: batch_x,
                model.char_ids: batch_char,
            },
    ))
    real = pred2label(batch_y)
    predict_Y.extend(predicted)
    real_Y.extend(real)

validation minibatch loop: 100%|██████████| 32/32 [00:09<00:00,  3.68it/s]


In [27]:
from sklearn.metrics import classification_report
print(classification_report(np.array(real_Y).ravel(), np.array(predict_Y).ravel(), digits = 6))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

         ADJ   0.998809  0.998333  0.998571      4199
         ADP   0.998502  0.999559  0.999030     11339
         ADV   0.996052  0.997088  0.996569      4807
         AUX   0.986081  1.000000  0.992992       921
       CCONJ   0.997556  0.999184  0.998370      3677
         DET   0.995592  0.996966  0.996278      3625
        NOUN   0.999645  0.998976  0.999311     25403
         NUM   0.998997  1.000000  0.999498      3984
        PART   0.994444  1.000000  0.997214       537
        PRON   0.999798  0.999798  0.999798      4941
       PROPN   0.999587  0.999633  0.999610     21776
       SCONJ   0.999316  0.993207  0.996252      1472
         SYM   1.000000  0.995951  0.997972       247
        VERB   0.999436  0.998711  0.999073     12411
           X   0.000000  0.000000  0.000000        11

    accuracy                       0.998863     99350
   macro avg   0.930921  0.931827  0.931369     99350
weighted avg   0.998754  0

In [28]:
text = 'Husein makan ayam dan ikan'
text_sample = np.array([text.split()])
char_test = generate_char_seq(text_sample)
word_test = word_seq(text_sample)
char_test.shape, word_test.shape

((1, 5, 6), (1, 5))

In [29]:
char_test

array([[[49,  5, 10, 13,  9, 11],
        [ 0,  3,  2, 25,  2, 11],
        [ 0,  0,  2, 24,  2,  3],
        [ 0,  0,  0,  7,  2, 11],
        [ 0,  0,  9, 25,  2, 11]]], dtype=int32)

In [30]:
predicted = pred2label(sess.run(model.tags_seq,
        feed_dict = {
            model.word_ids: word_test,
            model.char_ids: char_test,
        },
))
predicted

[['PROPN', 'VERB', 'NOUN', 'CCONJ', 'NOUN']]

In [31]:
text.split()

['Husein', 'makan', 'ayam', 'dan', 'ikan']